This script was used to transform the Gridded Anthropogenic Methane Emissions Inventory monthly dataset from netCDF to Cloud Optimized GeoTIFF (COG) format for display in the Greenhouse Gas (GHG) Center.


In [5]:
import os
import xarray
import re
import pandas as pd
import json
import tempfile
import boto3
from datetime import datetime
import numpy as np

from dotenv import load_dotenv

load_dotenv()

True

In [6]:
# session = boto3.session.Session()
session = boto3.Session(
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
    aws_session_token=os.environ.get("AWS_SESSION_TOKEN"),
)
s3_client = session.client("s3")
bucket_name = "ghgc-data-store-dev" # S3 bucket where the COGs are stored after transformation
FOLDER_NAME = "../data/epa_emissions_express_extension"
s3_folder_name = "epa_express_extension_Mg_km2_yr"
# raw gridded data [molec/cm2/s] * 1/6.022x10^23 [molec/mol] * 16.04x10^-6 [ Mg/mol] * 366 [days/yr] * 1x10^10 [cm2/km2]

files_processed = pd.DataFrame(columns=["file_name", "COGs_created"])   # A dataframe to keep track of the files that we have transformed into COGs

# Reading the raw netCDF files from local machine
for name in os.listdir(FOLDER_NAME):
    xds = xarray.open_dataset(f"{FOLDER_NAME}/{name}", engine="netcdf4")
    xds = xds.assign_coords(lon=(((xds.lon + 180) % 360) - 180)).sortby("lon")
    variable = [var for var in xds.data_vars]
    filename = name.split("/ ")[-1]
    filename_elements = re.split("[_ .]", filename)
    start_time = datetime(int(filename_elements[-2]), 1, 1)

    for time_increment in range(0, len(xds.time)):
        for var in variable:
            filename = name.split("/ ")[-1]
            filename_elements = re.split("[_ .]", filename)
            data = getattr(xds.isel(time=time_increment), var)
            data.values[data.values==0] = np.nan
            data = data*((1/(6.022*pow(10,23)))*(16.04*pow(10,-6))*366*pow(10,10)*86400)
            data = data.fillna(-9999)
            data = data.isel(lat=slice(None, None, -1))
            data.rio.set_spatial_dims("lon", "lat", inplace=True)
            data.rio.write_crs("epsg:4326", inplace=True)

            # # insert date of generated COG into filename
            filename_elements.pop()
            filename_elements[-1] = start_time.strftime("%Y")
            filename_elements.insert(2, var)
            cog_filename = "_".join(filename_elements)
            # # add extension
            cog_filename = f"{cog_filename}.tif"

            with tempfile.NamedTemporaryFile() as temp_file:
                data.rio.to_raster(
                    temp_file.name,
                    driver="COG",
                )
                s3_client.upload_file(
                    Filename=temp_file.name,
                    Bucket=bucket_name,
                    Key=f"{s3_folder_name}/{cog_filename}",
                )

            files_processed = files_processed._append(
                {"file_name": name, "COGs_created": cog_filename},
                ignore_index=True,
            )

            print(f"Generated and saved COG: {cog_filename}")

# Generate the json file with the metadata that is present in the netCDF files.
with tempfile.NamedTemporaryFile(mode="w+") as fp:
    json.dump(xds.attrs, fp)
    json.dump({"data_dimensions": dict(xds.dims)}, fp)
    json.dump({"data_variables": list(xds.data_vars)}, fp)
    fp.flush()

    s3_client.upload_file(
        Filename=fp.name,
        Bucket=bucket_name,
        Key=f"{s3_folder_name}/metadata.json",
    )

# creating the csv file with the names of files transformed.
files_processed.to_csv(
    f"s3://{bucket_name}/{s3_folder_name}/files_converted.csv",
)
print("Done generating COGs")


Generated and saved COG: Express_Extension_emi_ch4_1A_Combustion_Mobile_Gridded_GHGI_Methane_v2_2015.tif
Generated and saved COG: Express_Extension_emi_ch4_1A_Combustion_Stationary_Gridded_GHGI_Methane_v2_2015.tif
Generated and saved COG: Express_Extension_emi_ch4_1B1a_Abandoned_Coal_Gridded_GHGI_Methane_v2_2015.tif
Generated and saved COG: Express_Extension_emi_ch4_1B1a_Surface_Coal_Gridded_GHGI_Methane_v2_2015.tif
Generated and saved COG: Express_Extension_emi_ch4_1B1a_Underground_Coal_Gridded_GHGI_Methane_v2_2015.tif
Generated and saved COG: Express_Extension_emi_ch4_1B2a_Petroleum_Systems_Exploration_Gridded_GHGI_Methane_v2_2015.tif
Generated and saved COG: Express_Extension_emi_ch4_1B2a_Petroleum_Systems_Production_Gridded_GHGI_Methane_v2_2015.tif
Generated and saved COG: Express_Extension_emi_ch4_1B2a_Petroleum_Systems_Refining_Gridded_GHGI_Methane_v2_2015.tif
Generated and saved COG: Express_Extension_emi_ch4_1B2a_Petroleum_Systems_Transport_Gridded_GHGI_Methane_v2_2015.tif
Gene